In [49]:
import tensorflow as tf
from tensorflow import keras

In [50]:
tf.__version__

'2.3.0'

创建一个自定义的全连接层（dense）

In [51]:
class Linear(tf.keras.layers.Layer):
    def __init__(self, units=32,input_dim=32):
        super().__init__()
        self.units = units
        self.input_dim = input_dim

        self.w = self.add_variable(name='w',
            shape=[self.input_dim, self.units], initializer=tf.random_normal_initializer(),trainable=True)
        self.b = self.add_variable(name='b',
            shape=[self.units], initializer=tf.zeros_initializer(),trainable=True)

    def call(self, inputs):
        y_pred = tf.matmul(inputs, self.w) + self.b
        return y_pred

In [52]:
my_linear = Linear(4, 2)

In [53]:
my_linear.weights

[<tf.Variable 'w:0' shape=(2, 4) dtype=float32, numpy=
 array([[ 0.05543784, -0.07811399,  0.0345751 , -0.07654205],
        [ 0.00211201,  0.07324024, -0.05754722,  0.02302237]],
       dtype=float32)>,
 <tf.Variable 'b:0' shape=(4,) dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>]

In [54]:
x = tf.ones((2,2))

In [55]:
y = my_linear(x)

In [56]:
y

<tf.Tensor: shape=(2, 4), dtype=float32, numpy=
array([[ 0.05754986, -0.00487375, -0.02297212, -0.05351968],
       [ 0.05754986, -0.00487375, -0.02297212, -0.05351968]],
      dtype=float32)>

In [57]:
my_linear.weights

[<tf.Variable 'w:0' shape=(2, 4) dtype=float32, numpy=
 array([[ 0.05543784, -0.07811399,  0.0345751 , -0.07654205],
        [ 0.00211201,  0.07324024, -0.05754722,  0.02302237]],
       dtype=float32)>,
 <tf.Variable 'b:0' shape=(4,) dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>]

还可以使用一种更加快捷的方式为层添加权重：add_weight()方法：

In [58]:
class Linear(tf.keras.layers.Layer):
    def __init__(self, units=32,input_dim=32):
        super().__init__()
        self.units = units
        self.input_dim = input_dim

        self.w = self.add_weight(
            shape=(input_dim,units), 
            initializer='random_normal',
            trainable=True
        )
        self.b = self.add_weight(
            shape=(units,), 
            initializer='zeros',
            trainable=True
        )

    def call(self, inputs):
        y_pred = tf.matmul(inputs, self.w) + self.b
        return y_pred

In [59]:
x = tf.keras.layers.Dense(64)

In [60]:
x.weights

[]

将权重创建推迟到得知输入的形状之后<br></br>在层的build(self,input_shape)方法中创建层的权重

In [61]:
class Linear(tf.keras.layers.Layer):
    def __init__(self, units=32):
        super().__init__()
        self.units = units
        
    def build(self, input_shape):
        self.w = self.add_weight(
            shape=(input_shape[-1],self.units), 
            initializer='random_normal',
            trainable=True
        )
        self.b = self.add_weight(
            shape=(self.units,), 
            initializer='zeros',
            trainable=True
        )

    def call(self, inputs):
        y_pred = tf.matmul(inputs, self.w) + self.b
        return y_pred

In [62]:
my_linear = Linear(4)

In [63]:
my_linear.weights

[]

In [64]:
x = tf.ones((2,2))

In [65]:
my_linear(x)

<tf.Tensor: shape=(2, 4), dtype=float32, numpy=
array([[ 0.05428372, -0.07787265, -0.10486257, -0.06847197],
       [ 0.05428372, -0.07787265, -0.10486257, -0.06847197]],
      dtype=float32)>

In [66]:
my_linear.weights

[<tf.Variable 'linear_10/Variable:0' shape=(2, 4) dtype=float32, numpy=
 array([[ 0.12050934, -0.04452366, -0.07737461,  0.0138237 ],
        [-0.06622562, -0.03334899, -0.02748796, -0.08229566]],
       dtype=float32)>,
 <tf.Variable 'linear_10/Variable:0' shape=(4,) dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>]

层的call()方法将在首次调用时自动运行构建训练函数

### 层可递归组合

如果将一个层实例分配为另一个层的特性，则外部层将开始跟踪内部层的权重。

我们建议在init()方法中创建此类子层（由于子层通常具有构建方法，它们将与外部层同时构建）。

In [67]:
class MLPBlock(keras.layers.Layer):
    def __init__(self):
        super(MLPBlock, self).__init__()
        self.lin_1 = Linear(32)
        self.lin_2 = Linear(64)
        self.lin_3 = Linear(1)

    def call(self, inputs):
        x = self.lin_1(inputs)
        x = tf.nn.relu(x)
        x = self.lin_2(x)
        x = tf.nn.relu(x)
        x = self.lin_3(x)
        return x

In [68]:
# mlp = MLPBlock()

In [ ]:
mlp()

In [76]:
class MLPBlock_2(keras.layers.Layer):
    def __init__(self):
        super(MLPBlock_2, self).__init__()
        self.lin_1 = tf.keras.layers.Dense(32)
        self.lin_2 = tf.keras.layers.Dense(64)
        self.lin_3 = tf.keras.layers.Dense(32)

    def call(self, inputs):
        x1 = self.lin_1(inputs)
        x1 = tf.nn.relu(x1)
        x2 = self.lin_2(x1)
        x2 = tf.nn.relu(x2)
        x3 = self.lin_3(x2)
        return tf.concat([x1, x3])

In [77]:
mlp = MLPBlock_2()

In [78]:
mlp.fit

AttributeError: 'MLPBlock_2' object has no attribute 'fit'

通常使用Layer类来定义内部计算块，并使用Model类来定义外部模型，即您将训练的对象。

Model类具有与Layer相同的API，但有如下区别：

它会公开内置训练、评估和预测循环（model.fit()、model.evaluate()、model.predict())。

它会通过model.layer属性公开其内部层的列表。

它会公开保存和序列化API（save()、save_weights()……）

In [73]:
class MLP_model(keras.Model):
    def __init__(self):
        super(MLP_model, self).__init__()
        self.lin_1 = tf.keras.layers.Dense(32)
        self.lin_2 = tf.keras.layers.Dense(64)
        self.lin_3 = tf.keras.layers.Dense(32)

    def call(self, inputs):
        x1 = self.lin_1(inputs)
        x1 = tf.nn.relu(x1)
        x2 = self.lin_2(x1)
        x2 = tf.nn.relu(x2)
        x3 = self.lin_3(x2)
        return tf.concat([x1, x3])

In [74]:
model = MLP_model()

In [75]:
model.fit

<bound method Model.fit of <__main__.MLP_model object at 0x0000026734993B50>>

因此，如果您想知道“我应该用Layer类还是Model类？”，

请问自己：我是否需要在它上面调用fit()?

我是否需要在它上面调用save()?

如果是，则使用Model。如果不是（要么因为您的类只是更大系统中的一个块，要么因为您正在自己编写训练和保存代码），则使用Layer。